In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from xiao_utils import f

In [2]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)

In [5]:
a = df.groupby(['class_id','sale_date']).sum()['sale_quantity']

In [9]:
a = a.reset_index()

In [17]:
sale_201611 = pd.DataFrame(a[a['sale_date']==pd.to_datetime('201611', format="%Y%m")])

In [18]:
sale_201611['sale_201611'] = sale_201611['sale_quantity'] * 1.15

In [19]:
sale_201611.count()

class_id         118
sale_date        118
sale_quantity    118
sale_201611      118
dtype: int64

In [20]:
sale_201611

,class_id,sale_date,sale_quantity,sale_201611
20,103507,2016-11-01,876,1007.40
62,124140,2016-11-01,301,346.15
132,125403,2016-11-01,131,150.65
202,136916,2016-11-01,213,244.95
266,175962,2016-11-01,411,472.65
336,178529,2016-11-01,222,255.30
357,186250,2016-11-01,274,315.10
428,194450,2016-11-01,319,366.85
481,198427,2016-11-01,187,215.05
509,206765,2016-11-01,1592,1830.80


In [21]:
%qtconsole

In [35]:
s_1710 = pd.read_csv('../../result/Oct_mul_115percents.csv')

In [36]:
s1710 = s_1710.drop('predict_date', axis=1)

In [37]:
cf = sale_201611[['class_id','sale_201611']].rename(columns={'sale_201611':'predict_quantity'}).combine_first(other=s1710)

In [38]:
cf

,class_id,predict_quantity
0,103507.0,207.00
1,124140.0,302.00
2,125403.0,179.00
3,136916.0,186.00
4,169673.0,175.00
5,175962.0,273.00
6,178529.0,170.00
7,186250.0,95.00
8,194201.0,435.00
9,194450.0,312.00


In [39]:
sale_201611

,class_id,sale_date,sale_quantity,sale_201611
20,103507,2016-11-01,876,1007.40
62,124140,2016-11-01,301,346.15
132,125403,2016-11-01,131,150.65
202,136916,2016-11-01,213,244.95
266,175962,2016-11-01,411,472.65
336,178529,2016-11-01,222,255.30
357,186250,2016-11-01,274,315.10
428,194450,2016-11-01,319,366.85
481,198427,2016-11-01,187,215.05
509,206765,2016-11-01,1592,1830.80


In [30]:
s1710

,class_id,sale_quantity
0,103507,207.0
1,124140,302.0
2,125403,179.0
3,136916,186.0
4,169673,175.0
5,175962,273.0
6,178529,170.0
7,186250,95.0
8,194201,435.0
9,194450,312.0


In [45]:
s1611 = sale_201611[['class_id','sale_201611']].rename(columns={'sale_201611':'predict_quantity'})

In [43]:
res = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv')

In [74]:
merged = pd.merge(s1710,s1611, on='class_id', how='left')

In [75]:
merged

,class_id,predict_quantity_x,predict_quantity_y
0,103507,207.0,1007.40
1,124140,302.0,346.15
2,125403,179.0,150.65
3,136916,186.0,244.95
4,169673,175.0,NaN
5,175962,273.0,472.65
6,178529,170.0,255.30
7,186250,95.0,315.10
8,194201,435.0,NaN
9,194450,312.0,366.85


In [58]:
merged[merged['predict_quantity_y'].isnull()]['predict_quantity_y'] = merged[merged['predict_quantity_y'].isnull()]['predict_quantity_x']

C:\Users\xiao\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
merged.loc[merged['predict_quantity_y'].isnull(), 'predict_quantity_y'] = merged[merged['predict_quantity_y'].isnull()]['predict_quantity_x']

In [77]:
merged

,class_id,predict_quantity_x,predict_quantity_y
0,103507,207.0,1007.40
1,124140,302.0,346.15
2,125403,179.0,150.65
3,136916,186.0,244.95
4,169673,175.0,175.00
5,175962,273.0,472.65
6,178529,170.0,255.30
7,186250,95.0,315.10
8,194201,435.0,435.00
9,194450,312.0,366.85


In [65]:
# 求16年11月和17年10月销量（二者都乘以了1.15）的均值
merged['predict_quantity'] = merged[['predict_quantity_x', 'predict_quantity_y']].mean(1).round(0)

In [79]:
# 用16年11月销量乘以1.15，空缺的值用17年10月乘以1.15的值来填充，以此作为预测17年11月的
merged['predict_quantity'] = merged['predict_quantity_y']

In [80]:
res_1611_popwith_1710 = pd.merge(res.drop('predict_quantity', axis=1), merged[['class_id','predict_quantity']], on='class_id', how='left')

In [81]:
res_1611_popwith_1710.to_csv("../../result/16Nov_mul_115(fillwith_17Oct_mul_115).csv",index=False)

In [66]:
res_1611_1710 = pd.merge(res.drop('predict_quantity', axis=1), merged[['class_id','predict_quantity']], on='class_id', how='left')

In [67]:
res_1611_1710

,predict_date,class_id,predict_quantity
0,201711,103507,607.0
1,201711,124140,324.0
2,201711,125403,165.0
3,201711,136916,215.0
4,201711,169673,175.0
5,201711,175962,373.0
6,201711,178529,213.0
7,201711,186250,205.0
8,201711,194201,435.0
9,201711,194450,339.0


In [69]:
res_1611_1710.to_csv("../../result/1611_mul_1.15+1710_mul_1.15.csv",index=False)

In [48]:
tt = res[['class_id','predict_quantity']].combine_first(s1611)

In [50]:
tt['class_id'].value_counts()

559132.0    2
175962.0    2
948936.0    2
194450.0    2
628041.0    2
249875.0    2
136916.0    2
961362.0    2
651347.0    2
732758.0    2
961962.0    2
736094.0    2
516750.0    2
302513.0    2
350259.0    2
198427.0    2
360648.0    2
540712.0    2
379265.0    2
291514.0    2
819061.0    2
580470.0    2
501564.0    2
378536.0    2
649213.0    2
482497.0    2
178529.0    2
250658.0    2
560265.0    2
221795.0    2
           ..
591790.0    2
304458.0    2
786351.0    2
425432.0    2
714152.0    2
417803.0    1
653436.0    1
890189.0    1
854548.0    1
861459.0    1
340175.0    1
872180.0    1
103507.0    1
527765.0    1
815230.0    1
687270.0    1
359378.0    1
842246.0    1
169673.0    1
714150.0    1
725137.0    1
125403.0    1
905061.0    1
194201.0    1
612523.0    1
789290.0    1
124140.0    1
760412.0    1
437063.0    1
376193.0    1
Name: class_id, Length: 140, dtype: int64

In [ ]:

res['predict_quantity'] = pred_Nov
res.to_csv("../../result/v0.5.csv",index=False)